# Preprocessing

In [1]:
#content google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cat /proc/uptime | awk '{printf("残り時間 : %.2f", 12-$1/60/60)}'

残り時間 : 11.98

In [3]:
import tensorflow as tf
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import SGD
import tensorflow_hub as hub
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
!pip install tensorflow-addons
import tensorflow_addons as tfa
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score
import glob
#select tensorflow's version
%tensorflow_version 2.x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.4 MB/s 
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [4]:
#形状設定
IMAGE_SHAPE=(224,224)

#リアルタイムにデータ拡張しながら，テンソル画像データのバッチを生成する。(1/255 メモリ消費の削減)
image_generator=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3
    )

In [5]:
# HDF5フォーマットでモデルを保存するために必要
!pip install -q pyyaml h5py  

In [6]:
#各データセットのディレクトリを選択し、全画像をリサイズする。
#ディレクトリへのバスを受け取り、拡張・正規化したデータバッチを生成する。
batch_size=32

train_generator = image_generator.flow_from_directory(
    '/content/drive/MyDrive/data/train',
    target_size=(224, 224),
    #class_mode='binary',
    batch_size=batch_size,
    subset='training',
)

val_generator = image_generator.flow_from_directory(
    '/content/drive/MyDrive/data/train',
    target_size=(224, 224),
    #class_mode='binary',
    batch_size=batch_size,
    subset='validation'
)

Found 8400 images belonging to 4 classes.
Found 3600 images belonging to 4 classes.


In [7]:
#各データセットのディレクトリを選択し、全画像をリサイズする。
#ディレクトリへのバスを受け取り、拡張・正規化したデータバッチを生成する。
#train_data=image_generator.flow_from_directory('/content/drive/MyDrive/data/train',target_size=IMAGE_SHAPE)
test_data=image_generator.flow_from_directory('/content/drive/MyDrive/data/test',target_size=IMAGE_SHAPE)
gizi_data=image_generator.flow_from_directory('/content/drive/MyDrive/data/train',target_size=IMAGE_SHAPE)
val_data=image_generator.flow_from_directory('/content/drive/MyDrive/data/val',target_size=IMAGE_SHAPE)

Found 968 images belonging to 4 classes.
Found 12000 images belonging to 4 classes.
Found 4000 images belonging to 4 classes.


In [8]:
#各データセットの画像バッチ形状と、ラベルのバッチ形状を表示する
#擬似データはラベル付けない
for image_train_batch,label_train_batch in train_generator:
  print("Image batch shape:",image_train_batch.shape)
  print("Label batch shape:",label_train_batch.shape)
  break

for image_train_batch,label_train_batch in train_generator:
  print("Image batch shape:",image_train_batch.shape)
  print("Label batch shape:",label_train_batch.shape)
  break

for image_val_batch,label_val_batch in val_data:
  print("Image batch shape:",image_val_batch.shape)
  print("Label batch shape:",label_val_batch.shape)
  break

for image_test_batch,label_test_batch in test_data:
  print("Image batch shape:",image_test_batch.shape)
  print("Label batch shape:",label_test_batch.shape)
  break

Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 4)
Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 4)
Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 4)
Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 4)


In [9]:
#モデル設定
MODEL_DIR='/content/drive/MyDrive/result11'
if not os.path.exists(MODEL_DIR):
  os.makedirs(MODEL_DIR)

checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(MODEL_DIR,"model-{epoch:02d}.h5"),
                                              monitor='val_loss',
                                              mode='min',
                                              save_weights_only=True,
                                              verbose=1
                                              )
#filepath:モデルファイルを保存する場所を選択
#monitor:監視するメトリック名
#mode:ファイルを上書きするかどうか
#save_weights_only:モデルのウェイトのみを保存する
#verbose:1、冗長モード                                             

In [10]:
checkpoint1=tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(MODEL_DIR,"model01-{epoch:02d}.h5"),
                                              monitor='val_loss',
                                              mode='min',
                                              save_weights_only=True,
                                              verbose=1
                                              )

In [11]:
checkpoint2=tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(MODEL_DIR,"model02-{epoch:02d}.h5"),
                                              monitor='val_loss',
                                              mode='min',
                                              save_weights_only=True,
                                              verbose=1
                                              )

In [12]:
checkpoint3=tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(MODEL_DIR,"model03-{epoch:02d}.h5"),
                                              monitor='val_loss',
                                              mode='min',
                                              save_weights_only=True,
                                              verbose=1
                                              )

In [13]:
#訓練プロセスの可視化
#カスタムコールバックを作成して、すべてのエポックでのトレーニングの進行状況を視覚化します
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses=[]
    self.batch_acc=[]
    
  def on_train_batch_end(self,batch,logs=1e-12):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.model.reset_metrics()

In [14]:
#使用するモデルの選択（特徴抽出器/分類器）_ efficientnetv2
feature_extractor_url="https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_xl/feature_vector/2"

#入力画像サイズを決定する(モデルに適したサイズを設定)
feature_extractor_layer=hub.KerasLayer(feature_extractor_url,input_shape=(512,512,3))

#学習済み重みを固定する
feature_extractor_layer.trainable=False

#上位の分類層を接合する
ebc1=Sequential([feature_extractor_layer,layers.Dense(train_generator.num_classes,activation='softmax')])

#モデルの概要を確認する
ebc1.summary()

ebc1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense (Dense)               (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [15]:
#使用するモデルの選択（特徴抽出器/分類器）_ efficientnetb1
feature_extractor_url_b1="https://tfhub.dev/google/efficientnet/b1/feature-vector/1"

#入力画像サイズを決定する(モデルに適したサイズを設定)
feature_extractor_layer_b1=hub.KerasLayer(feature_extractor_url_b1,input_shape=(512,512,3))

#学習済み重みを固定する
feature_extractor_layer_b1.trainable=False

#上位の分類層を接合する
ebc2=Sequential([feature_extractor_layer_b1,layers.Dense(train_generator.num_classes,activation='softmax')])

#モデルの概要を確認する
ebc2.summary()

ebc2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              6575232   
                                                                 
 dense_1 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 6,580,356
Trainable params: 5,124
Non-trainable params: 6,575,232
_________________________________________________________________


In [16]:
#使用するモデルの選択（特徴抽出器/分類器）_ efficientnetb2
feature_extractor_url_b2="https://tfhub.dev/google/efficientnet/b2/feature-vector/1"

#入力画像サイズを決定する(モデルに適したサイズを設定)
feature_extractor_layer_b2=hub.KerasLayer(feature_extractor_url_b2,input_shape=(512,512,3))

#学習済み重みを固定する
feature_extractor_layer_b2.trainable=False

#上位の分類層を接合する
ebc3=Sequential([feature_extractor_layer_b2,layers.Dense(train_generator.num_classes,activation='softmax')])

#モデルの概要を確認する
ebc3.summary()

ebc3.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1408)              7768562   
                                                                 
 dense_2 (Dense)             (None, 4)                 5636      
                                                                 
Total params: 7,774,198
Trainable params: 5,636
Non-trainable params: 7,768,562
_________________________________________________________________


In [17]:
batch_stats_callback=CollectBatchStats()
batch_size=32
epochz=3
epoch=15
#steps_per_epoch
s_p_e=np.ceil(train_generator.samples/batch_size)

# model 1

model1

In [ ]:
ebc1.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

Epoch 1/3
265/265 [==============================] - 6921s 26s/step - loss: 0.4547 - acc: 0.8750
Epoch 2/3
265/265 [==============================] - 6931s 26s/step - loss: 0.2585 - acc: 0.8750
Epoch 3/3
265/265 [==============================] - 7135s 27s/step - loss: 0.2894 - acc: 0.9062


In [ ]:
ebc1.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

Epoch 1/3
265/265 [==============================] - 7915s 30s/step - loss: 0.1661 - acc: 0.9375
Epoch 2/3
265/265 [==============================] - 7450s 28s/step - loss: 0.2484 - acc: 0.9375
Epoch 3/3
265/265 [==============================] - 7092s 27s/step - loss: 0.1019 - acc: 0.9688


In [ ]:
ebc1.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

Epoch 1/3
265/265 [==============================] - 7240s 27s/step - loss: 0.3580 - acc: 0.8696
Epoch 2/3
265/265 [==============================] - 7554s 29s/step - loss: 0.1655 - acc: 0.9062
Epoch 3/3
265/265 [==============================] - 7566s 29s/step - loss: 0.2184 - acc: 0.8750


In [ ]:
ebc1.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

Epoch 1/3
265/265 [==============================] - 7114s 27s/step - loss: 0.3442 - acc: 0.8750
Epoch 2/3
265/265 [==============================] - 7013s 26s/step - loss: 0.2897 - acc: 0.8438
Epoch 3/3
265/265 [==============================] - 6962s 26s/step - loss: 0.2257 - acc: 0.8750


In [ ]:
ebc1.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

Epoch 1/3
265/265 [==============================] - 7014s 26s/step - loss: 0.1598 - acc: 0.9688
Epoch 2/3
265/265 [==============================] - 7031s 27s/step - loss: 0.4163 - acc: 0.9062
Epoch 3/3
265/265 [==============================] - 7097s 27s/step - loss: 0.1041 - acc: 0.9688


教師モデル学習

mark

In [ ]:
#上位分類層を接合
ebc1=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc1.summary()

ebc1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense_3 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [ ]:
ebc1.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[checkpoint1,batch_stats_callback],
         )

Epoch 1/3
265/265 [==============================] - ETA: 0s - loss: 0.2897 - acc: 0.8438 
Epoch 1: saving model to /content/drive/MyDrive/result11/model01-01.h5
265/265 [==============================] - 10209s 39s/step - loss: 0.2897 - acc: 0.8438 - val_loss: 0.5249 - val_acc: 0.8094
Epoch 2/3
265/265 [==============================] - ETA: 0s - loss: 0.2380 - acc: 0.9375 
Epoch 2: saving model to /content/drive/MyDrive/result11/model01-02.h5
265/265 [==============================] - 10196s 39s/step - loss: 0.2380 - acc: 0.9375 - val_loss: 0.5457 - val_acc: 0.7970
Epoch 3/3
265/265 [==============================] - ETA: 0s - loss: 0.1190 - acc: 1.0000 
Epoch 3: saving model to /content/drive/MyDrive/result11/model01-03.h5
265/265 [==============================] - 10235s 39s/step - loss: 0.1190 - acc: 1.0000 - val_loss: 0.4672 - val_acc: 0.8256


In [ ]:
ebc1.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[checkpoint1,batch_stats_callback],
         )

Epoch 1/3
265/265 [==============================] - ETA: 0s - loss: 0.6026 - acc: 0.7500 
Epoch 1: saving model to /content/drive/MyDrive/result11/model01-01.h5
265/265 [==============================] - 6639s 25s/step - loss: 0.6026 - acc: 0.7500 - val_loss: 0.5886 - val_acc: 0.7780
Epoch 2/3
265/265 [==============================] - ETA: 0s - loss: 0.5779 - acc: 0.8125 
Epoch 2: saving model to /content/drive/MyDrive/result11/model01-02.h5
265/265 [==============================] - 6606s 25s/step - loss: 0.5779 - acc: 0.8125 - val_loss: 0.4927 - val_acc: 0.8198
Epoch 3/3
265/265 [==============================] - ETA: 0s - loss: 0.1671 - acc: 1.0000 
Epoch 3: saving model to /content/drive/MyDrive/result11/model01-03.h5
265/265 [==============================] - 6664s 25s/step - loss: 0.1671 - acc: 1.0000 - val_loss: 0.4859 - val_acc: 0.8185


In [ ]:
ebc1.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[checkpoint1,batch_stats_callback],
         )

Epoch 1/3
265/265 [==============================] - ETA: 0s - loss: 0.0600 - acc: 1.0000 
Epoch 1: saving model to /content/drive/MyDrive/result11/model01-01.h5
265/265 [==============================] - 6748s 25s/step - loss: 0.0600 - acc: 1.0000 - val_loss: 0.4724 - val_acc: 0.8256
Epoch 2/3
265/265 [==============================] - ETA: 0s - loss: 0.1740 - acc: 0.9062 
Epoch 2: saving model to /content/drive/MyDrive/result11/model01-02.h5
265/265 [==============================] - 6826s 26s/step - loss: 0.1740 - acc: 0.9062 - val_loss: 0.4606 - val_acc: 0.8240
Epoch 3/3
265/265 [==============================] - ETA: 0s - loss: 0.1909 - acc: 0.9375 
Epoch 3: saving model to /content/drive/MyDrive/result11/model01-03.h5
265/265 [==============================] - 6715s 25s/step - loss: 0.1909 - acc: 0.9375 - val_loss: 0.5256 - val_acc: 0.8080


In [ ]:
ebc1.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[checkpoint1,batch_stats_callback],
         )

Epoch 1/3
265/265 [==============================] - ETA: 0s - loss: 0.4816 - acc: 0.7812 
Epoch 1: saving model to /content/drive/MyDrive/result11/model01-01.h5
265/265 [==============================] - 10370s 39s/step - loss: 0.4816 - acc: 0.7812 - val_loss: 0.5300 - val_acc: 0.7981
Epoch 2/3
265/265 [==============================] - ETA: 0s - loss: 0.4023 - acc: 0.8750 
Epoch 2: saving model to /content/drive/MyDrive/result11/model01-02.h5
265/265 [==============================] - 10455s 39s/step - loss: 0.4023 - acc: 0.8750 - val_loss: 0.5439 - val_acc: 0.7948
Epoch 3/3
265/265 [==============================] - ETA: 0s - loss: 0.2054 - acc: 0.9375 
Epoch 3: saving model to /content/drive/MyDrive/result11/model01-03.h5
265/265 [==============================] - 10709s 40s/step - loss: 0.2054 - acc: 0.9375 - val_loss: 0.4732 - val_acc: 0.8215


In [ ]:
ebc1.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[checkpoint1,batch_stats_callback],
         )

Epoch 1/3
265/265 [==============================] - ETA: 0s - loss: 0.1071 - acc: 1.0000 
Epoch 1: saving model to /content/drive/MyDrive/result11/model01-01.h5
265/265 [==============================] - 10552s 40s/step - loss: 0.1071 - acc: 1.0000 - val_loss: 0.4821 - val_acc: 0.8209
Epoch 2/3
265/265 [==============================] - ETA: 0s - loss: 0.2996 - acc: 0.9062 
Epoch 2: saving model to /content/drive/MyDrive/result11/model01-02.h5
265/265 [==============================] - 10481s 40s/step - loss: 0.2996 - acc: 0.9062 - val_loss: 0.4500 - val_acc: 0.8306
Epoch 3/3
265/265 [==============================] - ETA: 0s - loss: 0.2123 - acc: 0.9688 
Epoch 3: saving model to /content/drive/MyDrive/result11/model01-03.h5
265/265 [==============================] - 10611s 40s/step - loss: 0.2123 - acc: 0.9688 - val_loss: 0.4666 - val_acc: 0.8262


mark

In [ ]:
#上位分類層を接合
ebc1=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc1.summary()

ebc1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense_4 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [ ]:
#擬似ラベル作成
history=ebc1.fit(train_generator,
                  epochs=epochz,
                  steps_per_epoch=s_p_e,
                  validation_data=gizi_data,
                  callbacks=[checkpoint1,batch_stats_callback]
                  )

Epoch 1/3
265/265 [==============================] - ETA: 0s - loss: 0.5400 - acc: 0.7812 
Epoch 1: saving model to /content/drive/MyDrive/result11/model01-01.h5
265/265 [==============================] - 13663s 52s/step - loss: 0.5400 - acc: 0.7812 - val_loss: 0.4218 - val_acc: 0.8470
Epoch 2/3
265/265 [==============================] - ETA: 0s - loss: 0.2713 - acc: 0.8750 
Epoch 2: saving model to /content/drive/MyDrive/result11/model01-02.h5
265/265 [==============================] - 13793s 52s/step - loss: 0.2713 - acc: 0.8750 - val_loss: 0.3377 - val_acc: 0.8830
Epoch 3/3
265/265 [==============================] - ETA: 0s - loss: 0.2521 - acc: 0.9062 
Epoch 3: saving model to /content/drive/MyDrive/result11/model01-03.h5
265/265 [==============================] - 13845s 52s/step - loss: 0.2521 - acc: 0.9062 - val_loss: 0.3220 - val_acc: 0.8870


In [ ]:
history=ebc1.fit(train_generator,
                  epochs=epochz,
                  steps_per_epoch=s_p_e,
                  validation_data=gizi_data,
                  callbacks=[checkpoint1,batch_stats_callback]
                  )

Epoch 1/3
265/265 [==============================] - ETA: 0s - loss: 0.2781 - acc: 0.8750 
Epoch 1: saving model to /content/drive/MyDrive/result11/model01-01.h5
265/265 [==============================] - 13670s 52s/step - loss: 0.2781 - acc: 0.8750 - val_loss: 0.2886 - val_acc: 0.8965
Epoch 2/3
265/265 [==============================] - ETA: 0s - loss: 0.0784 - acc: 0.9688 
Epoch 2: saving model to /content/drive/MyDrive/result11/model01-02.h5
265/265 [==============================] - 14036s 53s/step - loss: 0.0784 - acc: 0.9688 - val_loss: 0.2820 - val_acc: 0.8998
Epoch 3/3
265/265 [==============================] - ETA: 0s - loss: 0.1907 - acc: 0.9375 
Epoch 3: saving model to /content/drive/MyDrive/result11/model01-03.h5
265/265 [==============================] - 14110s 53s/step - loss: 0.1907 - acc: 0.9375 - val_loss: 0.2877 - val_acc: 0.8990


In [ ]:
history=ebc1.fit(train_generator,
                  epochs=epochz,
                  steps_per_epoch=s_p_e,
                  validation_data=gizi_data,
                  callbacks=[checkpoint1,batch_stats_callback]
                  )

Epoch 1/3
 78/265 [=======>......................] - ETA: 1:23:59 - loss: 0.7705 - acc: 0.6875

In [ ]:
history=ebc1.fit(train_generator,
                  epochs=epochz,
                  steps_per_epoch=s_p_e,
                  validation_data=gizi_data,
                  callbacks=[checkpoint1,batch_stats_callback]
                  )

In [ ]:
history=ebc1.fit(train_generator,
                  epochs=epochz,
                  steps_per_epoch=s_p_e,
                  validation_data=gizi_data,
                  callbacks=[checkpoint1,batch_stats_callback]
                  )

In [ ]:
#疑似ラベルの予測
gizi_label1=ebc1.predict(gizi_data)

mark

In [ ]:
feature_extractor_layer.trainable=False
#入力画像ノイズ(rand augmentation)の設定
data_augmentation=Sequential([layers.RandomRotation(2.10)])
#モデルノイズ（stochastic depth）の設定
stochastic_Depth=Sequential([tfa.layers.StochasticDepth(0.35)])

#モデルにrand augmentationを適用
model1=Sequential([data_augmentation])
#モデルにstochastic depthを適用
model1=Sequential([stochastic_Depth])

#モデルノイズ（dropout）の設定と適用
model1=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax'),
                           #layers.Dropout(0.2)
                           
                  ])

mark

In [ ]:
#model1
model1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])
model1.summary()


In [ ]:
#model1
history=model1.fit(gizi_data,
                  epochs=epochz,
                  steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                  validation_data=val_generator,
                  callbacks=[checkpoint1,batch_stats_callback]
                  )

In [ ]:
history=model1.fit(gizi_data,
                  epochs=epochz,
                  steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                  validation_data=val_generator,
                  callbacks=[checkpoint1,batch_stats_callback]
                  )

In [ ]:
history=model1.fit(gizi_data,
                  epochs=epochz,
                  steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                  validation_data=val_generator,
                  callbacks=[checkpoint1,batch_stats_callback]
                  )

In [ ]:
history=model1.fit(gizi_data,
                  epochs=epochz,
                  steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                  validation_data=val_generator,
                  callbacks=[checkpoint1,batch_stats_callback]
                  )

In [ ]:
history=model1.fit(gizi_data,
                  epochs=epochz,
                  steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                  validation_data=val_generator,
                  callbacks=[checkpoint1,batch_stats_callback]
                  )

1

In [ ]:
history=model1.fit(train_generator,
                   epochs=epochz,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[checkpoint1,batch_stats_callback]
                   )

In [ ]:
history=model1.fit(train_generator,
                   epochs=epochz,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[checkpoint1,batch_stats_callback]
                   )

In [ ]:
history=model1.fit(train_generator,
                   epochs=epochz,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[checkpoint1,batch_stats_callback]
                   )

In [ ]:
history=model1.fit(train_generator,
                   epochs=epochz,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[checkpoint1,batch_stats_callback]
                   )

In [ ]:
history=model1.fit(train_generator,
                   epochs=epochz,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[checkpoint1,batch_stats_callback]
                   )

In [ ]:
# HDF5フォーマットでモデルを保存するために必要
!pip install -q pyyaml h5py  

In [ ]:
model1.save('/content/drive/MyDrive/result11/model1.h5/')

# model 2

model2

In [ ]:
ebc2.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

In [ ]:
ebc2.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

In [ ]:
ebc2.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

In [ ]:
ebc2.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

In [ ]:
ebc2.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

教師モデル学習

mark

In [ ]:
#上位分類層を接合
ebc2=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc2.summary()

ebc2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
ebc2.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[batch_stats_callback,checkpoint2],
         )

In [ ]:
ebc2.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[batch_stats_callback,checkpoint2],
         )

In [ ]:
ebc2.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[batch_stats_callback,checkpoint2],
         )

In [ ]:
ebc2.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[batch_stats_callback,checkpoint2],
         )

In [ ]:
ebc2.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[batch_stats_callback,checkpoint2],
         )

mark

In [ ]:
#上位分類層を接合
ebc2=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc2.summary()

ebc2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
#擬似ラベル作成
history=ebc2.fit(train_generator,
                 epochs=epochz,
                 steps_per_epoch=s_p_e,
                 validation_data=gizi_data,
                 callbacks=[checkpoint2,batch_stats_callback]
                 )

In [ ]:
history=ebc2.fit(train_generator,
                 epochs=epochz,
                 steps_per_epoch=s_p_e,
                 validation_data=gizi_data,
                 callbacks=[checkpoint2,batch_stats_callback]
                 )

In [ ]:
history=ebc2.fit(train_generator,
                 epochs=epochz,
                 steps_per_epoch=s_p_e,
                 validation_data=gizi_data,
                 callbacks=[checkpoint2,batch_stats_callback]
                 )

In [ ]:
history=ebc2.fit(train_generator,
                 epochs=epochz,
                 steps_per_epoch=s_p_e,
                 validation_data=gizi_data,
                 callbacks=[checkpoint2,batch_stats_callback]
                 )

In [ ]:
history=ebc2.fit(train_generator,
                 epochs=epochz,
                 steps_per_epoch=s_p_e,
                 validation_data=gizi_data,
                 callbacks=[checkpoint2,batch_stats_callback]
                 )

In [ ]:
gizi_label2=ebc2.predict(gizi_data)

mark

In [ ]:
feature_extractor_layer.trainable=False
#入力画像ノイズ(rand augmentation)の設定
data_augmentation=Sequential([layers.RandomRotation(2.10)])
#モデルノイズ（stochastic depth）の設定
stochastic_Depth=Sequential([tfa.layers.StochasticDepth(0.35)])

#モデルにrand augmentationを適用
model2=Sequential([data_augmentation])
#モデルにstochastic depthを適用
model2=Sequential([stochastic_Depth])



model2=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax'),
                           #layers.Dropout(0.2)
                  ])



mark

In [ ]:
#model2
model2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])
model2.summary()


In [ ]:
#model2
history=model2.fit(gizi_data,
                   epochs=epochz,
                   steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                   validation_data=val_generator,
                   callbacks=[checkpoint2,batch_stats_callback]
                   )


In [ ]:
history=model2.fit(gizi_data,
                   epochs=epochz,
                   steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                   validation_data=val_generator,
                   callbacks=[checkpoint2,batch_stats_callback]
                   )

In [ ]:
history=model2.fit(gizi_data,
                   epochs=epochz,
                   steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                   validation_data=val_generator,
                   callbacks=[checkpoint2,batch_stats_callback]
                   )

In [ ]:
history=model2.fit(gizi_data,
                   epochs=epochz,
                   steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                   validation_data=val_generator,
                   callbacks=[checkpoint2,batch_stats_callback]
                   )

In [ ]:
history=model2.fit(gizi_data,
                   epochs=epochz,
                   steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                   validation_data=val_generator,
                   callbacks=[checkpoint2,batch_stats_callback]
                   )

2

In [ ]:
history=model2.fit(train_generator,
                   epochs=epochz,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[checkpoint2,batch_stats_callback]
                   )

In [ ]:
history=model2.fit(train_generator,
                   epochs=epochz,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[checkpoint2,batch_stats_callback]
                   )

In [ ]:
history=model2.fit(train_generator,
                   epochs=epochz,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[checkpoint2,batch_stats_callback]
                   )

In [ ]:
history=model2.fit(train_generator,
                   epochs=epochz,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[checkpoint2,batch_stats_callback]
                   )

In [ ]:
history=model2.fit(train_generator,
                   epochs=epochz,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[checkpoint2,batch_stats_callback]
                   )

In [ ]:
model2.save('/content/drive/MyDrive/result11/model2.h5/')

# model 3

model3

In [ ]:
ebc3.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

Epoch 1/3
263/263 [==============================] - 1822s 7s/step - loss: 0.4137 - acc: 0.8750
Epoch 2/3
263/263 [==============================] - 792s 3s/step - loss: 0.4523 - acc: 0.8125
Epoch 3/3
263/263 [==============================] - 776s 3s/step - loss: 0.5737 - acc: 0.8125


In [ ]:
ebc3.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

Epoch 1/3
263/263 [==============================] - 776s 3s/step - loss: 0.3946 - acc: 0.9062
Epoch 2/3
263/263 [==============================] - 779s 3s/step - loss: 0.4160 - acc: 0.8750
Epoch 3/3
263/263 [==============================] - 773s 3s/step - loss: 0.3755 - acc: 0.8438


In [ ]:
ebc3.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

Epoch 1/3
263/263 [==============================] - 774s 3s/step - loss: 0.1617 - acc: 0.9688
Epoch 2/3
263/263 [==============================] - 779s 3s/step - loss: 0.1775 - acc: 0.9375
Epoch 3/3
263/263 [==============================] - 798s 3s/step - loss: 0.1674 - acc: 0.9688


In [ ]:
ebc3.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

Epoch 1/3
263/263 [==============================] - 822s 3s/step - loss: 0.1493 - acc: 0.9688
Epoch 2/3
263/263 [==============================] - 820s 3s/step - loss: 0.3806 - acc: 0.8750
Epoch 3/3
263/263 [==============================] - 822s 3s/step - loss: 0.1521 - acc: 0.9375


In [ ]:
ebc3.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

Epoch 1/3
263/263 [==============================] - 818s 3s/step - loss: 0.1138 - acc: 0.9375
Epoch 2/3
263/263 [==============================] - 816s 3s/step - loss: 0.2588 - acc: 0.8438
Epoch 3/3
263/263 [==============================] - 816s 3s/step - loss: 0.1272 - acc: 1.0000


mark

In [18]:
#上位分類層を接合
ebc3=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc3.summary()

ebc3.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense_3 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


教師モデル作成

In [ ]:
ebc3.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[batch_stats_callback,checkpoint3],
         )

Epoch 1/3
263/263 [==============================] - ETA: 0s - loss: 0.6925 - acc: 0.6875 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 10133s 38s/step - loss: 0.6925 - acc: 0.6875 - val_loss: 0.5169 - val_acc: 0.8053
Epoch 2/3
263/263 [==============================] - ETA: 0s - loss: 0.2176 - acc: 0.9375 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
263/263 [==============================] - 10072s 38s/step - loss: 0.2176 - acc: 0.9375 - val_loss: 0.4891 - val_acc: 0.8144
Epoch 3/3
263/263 [==============================] - ETA: 0s - loss: 0.1997 - acc: 0.9062 

In [ ]:
ebc3.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[batch_stats_callback,checkpoint3],
         )

Epoch 1/3
263/263 [==============================] - ETA: 0s - loss: 0.2319 - acc: 0.9375 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 10606s 40s/step - loss: 0.2319 - acc: 0.9375 - val_loss: 0.5999 - val_acc: 0.7706
Epoch 2/3
263/263 [==============================] - ETA: 0s - loss: 0.4577 - acc: 0.8125 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
263/263 [==============================] - 10710s 41s/step - loss: 0.4577 - acc: 0.8125 - val_loss: 0.4931 - val_acc: 0.8153
Epoch 3/3
263/263 [==============================] - ETA: 0s - loss: 0.2565 - acc: 0.9062 
Epoch 3: saving model to /content/drive/MyDrive/result11/model03-03.h5
263/263 [==============================] - 10697s 41s/step - loss: 0.2565 - acc: 0.9062 - val_loss: 0.4598 - val_acc: 0.8256


In [ ]:
ebc3.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[batch_stats_callback,checkpoint3],
         )

Epoch 1/3
263/263 [==============================] - ETA: 0s - loss: 0.3732 - acc: 0.8750 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 10236s 39s/step - loss: 0.3732 - acc: 0.8750 - val_loss: 0.5480 - val_acc: 0.7906
Epoch 2/3
263/263 [==============================] - ETA: 0s - loss: 0.2735 - acc: 0.9375 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
263/263 [==============================] - 10288s 39s/step - loss: 0.2735 - acc: 0.9375 - val_loss: 0.4757 - val_acc: 0.8194
Epoch 3/3
263/263 [==============================] - ETA: 0s - loss: 0.2295 - acc: 0.9375 
Epoch 3: saving model to /content/drive/MyDrive/result11/model03-03.h5
263/263 [==============================] - 10278s 39s/step - loss: 0.2295 - acc: 0.9375 - val_loss: 0.5372 - val_acc: 0.8033


In [ ]:
ebc3.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[batch_stats_callback,checkpoint3],
         )

Epoch 1/3
263/263 [==============================] - ETA: 0s - loss: 0.4783 - acc: 0.7500 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 10153s 38s/step - loss: 0.4783 - acc: 0.7500 - val_loss: 0.5378 - val_acc: 0.7875
Epoch 2/3
263/263 [==============================] - ETA: 0s - loss: 0.3639 - acc: 0.8750 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
263/263 [==============================] - 10183s 39s/step - loss: 0.3639 - acc: 0.8750 - val_loss: 0.4892 - val_acc: 0.8169
Epoch 3/3
263/263 [==============================] - ETA: 0s - loss: 0.1551 - acc: 0.9688 
Epoch 3: saving model to /content/drive/MyDrive/result11/model03-03.h5
263/263 [==============================] - 10220s 39s/step - loss: 0.1551 - acc: 0.9688 - val_loss: 0.4864 - val_acc: 0.8167


In [ ]:
ebc3.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[batch_stats_callback,checkpoint3],
         )

Epoch 1/3
263/263 [==============================] - ETA: 0s - loss: 0.4777 - acc: 0.8438 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 9098s 34s/step - loss: 0.4777 - acc: 0.8438 - val_loss: 0.5398 - val_acc: 0.7917
Epoch 2/3
263/263 [==============================] - ETA: 0s - loss: 0.3228 - acc: 0.8438 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
263/263 [==============================] - 9102s 35s/step - loss: 0.3228 - acc: 0.8438 - val_loss: 0.4558 - val_acc: 0.8294
Epoch 3/3
263/263 [==============================] - ETA: 0s - loss: 0.3232 - acc: 0.8750 
Epoch 3: saving model to /content/drive/MyDrive/result11/model03-03.h5
263/263 [==============================] - 9315s 35s/step - loss: 0.3232 - acc: 0.8750 - val_loss: 0.6212 - val_acc: 0.7747


mark

In [19]:
#上位分類層を接合
ebc3=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc3.summary()

ebc3.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense_4 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [ ]:
#擬似ラベル作成
history=ebc3.fit(train_generator,
                 epochs=epochz,
                 steps_per_epoch=s_p_e,
                 validation_data=gizi_data,
                 callbacks=[batch_stats_callback,checkpoint3],
                 )

Epoch 1/3
263/263 [==============================] - ETA: 0s - loss: 0.2463 - acc: 0.9375 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 15094s 57s/step - loss: 0.2463 - acc: 0.9375 - val_loss: 0.4156 - val_acc: 0.8483
Epoch 2/3
263/263 [==============================] - ETA: 0s - loss: 0.3041 - acc: 0.9062 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
263/263 [==============================] - 15418s 59s/step - loss: 0.3041 - acc: 0.9062 - val_loss: 0.3397 - val_acc: 0.8773
Epoch 3/3
263/263 [==============================] - ETA: 0s - loss: 0.1621 - acc: 0.9375 

In [ ]:
history=ebc3.fit(train_generator,
                 epochs=epochz,
                 steps_per_epoch=s_p_e,
                 validation_data=gizi_data,
                 callbacks=[batch_stats_callback,checkpoint3],
                 )

Epoch 1/3
263/263 [==============================] - ETA: 0s - loss: 0.4549 - acc: 0.8750 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 17663s 67s/step - loss: 0.4549 - acc: 0.8750 - val_loss: 0.4027 - val_acc: 0.8579
Epoch 2/3
263/263 [==============================] - ETA: 0s - loss: 0.2097 - acc: 0.9688 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
263/263 [==============================] - 17667s 67s/step - loss: 0.2097 - acc: 0.9688 - val_loss: 0.3556 - val_acc: 0.8741
Epoch 3/3
263/263 [==============================] - ETA: 0s - loss: 0.3888 - acc: 0.8750 

In [ ]:
history=ebc3.fit(train_generator,
                 epochs=epochz,
                 steps_per_epoch=s_p_e,
                 validation_data=gizi_data,
                 callbacks=[batch_stats_callback,checkpoint3],
                 )

Epoch 1/3
263/263 [==============================] - ETA: 0s - loss: 0.3555 - acc: 0.8750 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 17339s 66s/step - loss: 0.3555 - acc: 0.8750 - val_loss: 0.3939 - val_acc: 0.8640
Epoch 2/3
263/263 [==============================] - ETA: 0s - loss: 0.3370 - acc: 0.8438 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
263/263 [==============================] - 17512s 67s/step - loss: 0.3370 - acc: 0.8438 - val_loss: 0.3376 - val_acc: 0.8819
Epoch 3/3
263/263 [==============================] - ETA: 0s - loss: 0.2352 - acc: 0.9375 

In [ ]:
history=ebc3.fit(train_generator,
                 epochs=epochz,
                 steps_per_epoch=s_p_e,
                 validation_data=gizi_data,
                 callbacks=[batch_stats_callback,checkpoint3],
                 )

Epoch 1/3
263/263 [==============================] - ETA: 0s - loss: 0.3831 - acc: 0.8438 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 16072s 61s/step - loss: 0.3831 - acc: 0.8438 - val_loss: 0.4124 - val_acc: 0.8495
Epoch 2/3
263/263 [==============================] - ETA: 0s - loss: 0.2724 - acc: 0.9062 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
263/263 [==============================] - 15932s 61s/step - loss: 0.2724 - acc: 0.9062 - val_loss: 0.3294 - val_acc: 0.8838
Epoch 3/3
263/263 [==============================] - ETA: 0s - loss: 0.2163 - acc: 0.9062 

In [ ]:
history=ebc3.fit(train_generator,
                 epochs=epochz,
                 steps_per_epoch=s_p_e,
                 validation_data=gizi_data,
                 callbacks=[batch_stats_callback,checkpoint3],
                 )

Epoch 1/3
263/263 [==============================] - ETA: 0s - loss: 0.2432 - acc: 0.9688 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 16194s 62s/step - loss: 0.2432 - acc: 0.9688 - val_loss: 0.3825 - val_acc: 0.8661
Epoch 2/3
263/263 [==============================] - ETA: 0s - loss: 0.3231 - acc: 0.8750 

In [ ]:
gizi_label3=ebc3.predict(gizi_data)

375/375 [==============================] - 9999s 27s/step


mark

In [20]:
feature_extractor_layer.trainable=False
#入力画像ノイズ(rand augmentation)の設定
data_augmentation=Sequential([layers.RandomRotation(2.10)])
#モデルノイズ（stochastic depth）の設定
stochastic_Depth=Sequential([tfa.layers.StochasticDepth(0.35)])

#モデルにrand augmentationを適用
model3=Sequential([data_augmentation])
#モデルにstochastic depthを適用
model3=Sequential([stochastic_Depth])

model3=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax'),
                           #layers.Dropout(0.2)
                  ])

mark

In [21]:
#model3
model3.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])
model3.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense_5 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [ ]:
#model3
history=model3.fit(gizi_data,
                   epochs=epochz,
                   steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                   validation_data=val_generator,
                   callbacks=[checkpoint3,batch_stats_callback]
                   )

Epoch 1/3
375/375 [==============================] - ETA: 0s - loss: 0.3299 - acc: 0.8750 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
375/375 [==============================] - 12972s 35s/step - loss: 0.3299 - acc: 0.8750 - val_loss: 0.4810 - val_acc: 0.8158
Epoch 2/3
375/375 [==============================] - ETA: 0s - loss: 0.3182 - acc: 0.8438 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
375/375 [==============================] - 13059s 35s/step - loss: 0.3182 - acc: 0.8438 - val_loss: 0.3783 - val_acc: 0.8578
Epoch 3/3
375/375 [==============================] - ETA: 0s - loss: 0.1659 - acc: 0.9688 
Epoch 3: saving model to /content/drive/MyDrive/result11/model03-03.h5
375/375 [==============================] - 13198s 35s/step - loss: 0.1659 - acc: 0.9688 - val_loss: 0.3018 - val_acc: 0.8922


In [ ]:
history=model3.fit(gizi_data,
                   epochs=epochz,
                   steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                   validation_data=val_generator,
                   callbacks=[checkpoint3,batch_stats_callback]
                   )

Epoch 1/3
375/375 [==============================] - ETA: 0s - loss: 0.3126 - acc: 0.8750 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
375/375 [==============================] - 13344s 35s/step - loss: 0.3126 - acc: 0.8750 - val_loss: 0.4373 - val_acc: 0.8281
Epoch 2/3
375/375 [==============================] - ETA: 0s - loss: 0.2121 - acc: 0.9062 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
375/375 [==============================] - 13399s 36s/step - loss: 0.2121 - acc: 0.9062 - val_loss: 0.3354 - val_acc: 0.8753
Epoch 3/3
375/375 [==============================] - ETA: 0s - loss: 0.0914 - acc: 1.0000 
Epoch 3: saving model to /content/drive/MyDrive/result11/model03-03.h5
375/375 [==============================] - 13364s 36s/step - loss: 0.0914 - acc: 1.0000 - val_loss: 0.3260 - val_acc: 0.8708


In [ ]:
history=model3.fit(gizi_data,
                   epochs=epochz,
                   steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                   validation_data=val_generator,
                   callbacks=[checkpoint3,batch_stats_callback]
                   )

Epoch 1/3
375/375 [==============================] - ETA: 0s - loss: 0.4830 - acc: 0.8750 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
375/375 [==============================] - 12903s 34s/step - loss: 0.4830 - acc: 0.8750 - val_loss: 0.3824 - val_acc: 0.8586
Epoch 2/3
375/375 [==============================] - ETA: 0s - loss: 0.1879 - acc: 0.9688 

In [ ]:
history=model3.fit(gizi_data,
                   epochs=epochz,
                   steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                   validation_data=val_generator,
                   callbacks=[checkpoint3,batch_stats_callback]
                   )

Epoch 1/3
375/375 [==============================] - ETA: 0s - loss: 0.1904 - acc: 0.9375 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
375/375 [==============================] - 8659s 23s/step - loss: 0.1904 - acc: 0.9375 - val_loss: 0.3994 - val_acc: 0.8533
Epoch 2/3
375/375 [==============================] - ETA: 0s - loss: 0.3278 - acc: 0.8438 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
375/375 [==============================] - 8626s 23s/step - loss: 0.3278 - acc: 0.8438 - val_loss: 0.3842 - val_acc: 0.8589
Epoch 3/3
374/375 [============================>.] - ETA: 18s - loss: 0.3554 - acc: 0.8438

In [ ]:
history=model3.fit(gizi_data,
                   epochs=epochz,
                   steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                   validation_data=val_generator,
                   callbacks=[checkpoint3,batch_stats_callback]
                   )

Epoch 1/3
375/375 [==============================] - ETA: 0s - loss: 0.4397 - acc: 0.7812 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
375/375 [==============================] - 13213s 35s/step - loss: 0.4397 - acc: 0.7812 - val_loss: 0.3992 - val_acc: 0.8550
Epoch 2/3
375/375 [==============================] - ETA: 0s - loss: 0.1282 - acc: 1.0000 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
375/375 [==============================] - 13440s 36s/step - loss: 0.1282 - acc: 1.0000 - val_loss: 0.3354 - val_acc: 0.8803
Epoch 3/3
375/375 [==============================] - ETA: 0s - loss: 0.3605 - acc: 0.8438 
Epoch 3: saving model to /content/drive/MyDrive/result11/model03-03.h5
375/375 [==============================] - 13305s 35s/step - loss: 0.3605 - acc: 0.8438 - val_loss: 0.3051 - val_acc: 0.8906


3

In [ ]:
history=model3.fit(train_generator,
                   epochs=epochz,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[batch_stats_callback,checkpoint3]
                   )

Epoch 1/3
263/263 [==============================] - ETA: 0s - loss: 0.3940 - acc: 0.8750 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 10059s 38s/step - loss: 0.3940 - acc: 0.8750 - val_loss: 0.5151 - val_acc: 0.7992
Epoch 2/3
263/263 [==============================] - ETA: 0s - loss: 0.2473 - acc: 0.9062 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
263/263 [==============================] - 10222s 39s/step - loss: 0.2473 - acc: 0.9062 - val_loss: 0.5056 - val_acc: 0.8056
Epoch 3/3
263/263 [==============================] - ETA: 0s - loss: 0.1412 - acc: 0.9688 
Epoch 3: saving model to /content/drive/MyDrive/result11/model03-03.h5
263/263 [==============================] - 10283s 39s/step - loss: 0.1412 - acc: 0.9688 - val_loss: 0.4605 - val_acc: 0.8286


In [ ]:
history=model3.fit(train_generator,
                   epochs=2,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[batch_stats_callback,checkpoint3]
                   )

Epoch 1/2
263/263 [==============================] - ETA: 0s - loss: 0.3700 - acc: 0.8125 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 9959s 38s/step - loss: 0.3700 - acc: 0.8125 - val_loss: 0.5035 - val_acc: 0.8156
Epoch 2/2
263/263 [==============================] - ETA: 0s - loss: 0.2804 - acc: 0.9062 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
263/263 [==============================] - 10092s 38s/step - loss: 0.2804 - acc: 0.9062 - val_loss: 0.4839 - val_acc: 0.8144


In [ ]:
history=model3.fit(train_generator,
                   epochs=2,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[batch_stats_callback,checkpoint3]
                   )

Epoch 1/2
263/263 [==============================] - ETA: 0s - loss: 0.3036 - acc: 0.8438 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 10087s 38s/step - loss: 0.3036 - acc: 0.8438 - val_loss: 0.5864 - val_acc: 0.7733
Epoch 2/2
263/263 [==============================] - ETA: 0s - loss: 0.4611 - acc: 0.8750 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
263/263 [==============================] - 10281s 39s/step - loss: 0.4611 - acc: 0.8750 - val_loss: 0.5020 - val_acc: 0.8072


In [ ]:
history=model3.fit(train_generator,
                   epochs=2,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[batch_stats_callback,checkpoint3]
                   )

Epoch 1/2
263/263 [==============================] - ETA: 0s - loss: 0.3113 - acc: 0.9062 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 8973s 34s/step - loss: 0.3113 - acc: 0.9062 - val_loss: 0.6167 - val_acc: 0.7581
Epoch 2/2
263/263 [==============================] - ETA: 0s - loss: 0.3526 - acc: 0.8438 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
263/263 [==============================] - 9752s 37s/step - loss: 0.3526 - acc: 0.8438 - val_loss: 0.4752 - val_acc: 0.8164


In [ ]:
history=model3.fit(train_generator,
                   epochs=2,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[batch_stats_callback,checkpoint3]
                   )

Epoch 1/2
263/263 [==============================] - ETA: 0s - loss: 0.1146 - acc: 1.0000 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 10023s 38s/step - loss: 0.1146 - acc: 1.0000 - val_loss: 0.5580 - val_acc: 0.7900
Epoch 2/2
263/263 [==============================] - ETA: 0s - loss: 0.2580 - acc: 0.9375 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
263/263 [==============================] - 10711s 41s/step - loss: 0.2580 - acc: 0.9375 - val_loss: 0.4656 - val_acc: 0.8264


In [ ]:
history=model3.fit(train_generator,
                   epochs=2,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[batch_stats_callback,checkpoint3]
                   )

Epoch 1/2
263/263 [==============================] - ETA: 0s - loss: 0.2610 - acc: 0.9375 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 10167s 39s/step - loss: 0.2610 - acc: 0.9375 - val_loss: 0.5002 - val_acc: 0.8117
Epoch 2/2
263/263 [==============================] - ETA: 0s - loss: 0.4704 - acc: 0.8438 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
263/263 [==============================] - 10476s 40s/step - loss: 0.4704 - acc: 0.8438 - val_loss: 0.4686 - val_acc: 0.8261


In [22]:
history=model3.fit(train_generator,
                   epochs=2,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[batch_stats_callback,checkpoint3]
                   )

Epoch 1/2
263/263 [==============================] - ETA: 0s - loss: 0.2620 - acc: 0.9062 
Epoch 1: saving model to /content/drive/MyDrive/result11/model03-01.h5
263/263 [==============================] - 10049s 38s/step - loss: 0.2620 - acc: 0.9062 - val_loss: 0.5188 - val_acc: 0.8086
Epoch 2/2
263/263 [==============================] - ETA: 0s - loss: 0.2067 - acc: 0.9375 
Epoch 2: saving model to /content/drive/MyDrive/result11/model03-02.h5
263/263 [==============================] - 10013s 38s/step - loss: 0.2067 - acc: 0.9375 - val_loss: 0.5008 - val_acc: 0.8169


In [23]:
#モデル保存
model3.save('/content/drive/MyDrive/result11/model3.h5/')

# VotingClassifier

Accuracy for each model

In [ ]:
#content google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
model1 = keras.models.load_model('/content/drive/MyDrive/result11/model1.h5/')
model2 = keras.models.load_model('/content/drive/MyDrive/result11/model2.h5/')
model3 = keras.models.load_model('/content/drive/MyDrive/result11/model3.h5/')

In [ ]:
train_datagenerator = ImageDataGenerator(rescale = 1.0/255,
                                        shear_range = 0.2,
                                        zoom_range = 0.5,
                                        horizontal_flip = True,
                                        rotation_range=10,
                                        width_shift_range=0.2,
                                        brightness_range=[0.2,1.2]
                                        )
test_datagenerator = ImageDataGenerator(rescale = 1.0/255)

training_set = train_datagenerator.flow_from_directory('/content/drive/MyDrive/data/train/',
                                                    target_size = (224,224),
                                                    batch_size = 32,
                                                    class_mode = 'categorical')

class_dict = training_set.class_indices
print(class_dict)

li = list(class_dict.keys())
print(li)

valid_set = test_datagenerator.flow_from_directory('/content/drive/MyDrive/data/test',
                                                  target_size = (512,512),
                                                    batch_size = 32,
                                                    class_mode = 'categorical')

# valid_set
x_test, y_test = valid_set.next() 

In [ ]:
# predicting an image
import matplotlib.pyplot as plt
from keras.preprocessing import image
import numpy as np

class votingClassifer:
    'votingClassifer class'


    def __init__(self, estimators,mode="hard",weight=None,show_info="percent"):
        
        if mode not in ["hard","soft"]:
            raise Exception("the mode should be 'hard' or 'soft'")
            
        self.estimators=estimators
        self.mode=mode
        self.weight=weight
        self.show_info=show_info  
            
    def predict(self,x_test):        
        if self.mode=="hard":
            return self.votingClassifer_hard(self.estimators, x_test, show_info=self.show_info)
        else:
            return self.votingClassifer_soft(self.estimators, x_test, weight=self.weight, show_info=self.show_info)

    def getNumberDiff(self, index_classes, n):  
        
        for x in index_classes:
            if index_class_prefer != x:
                return x    
    
    def getNumberElse(self, index_classes, n):

        indices = [i for i, value in enumerate(index_classes) if value != n]
        counts = np.bincount(indices)
        ind=np.argmax(counts)
        n1=(indices == ind).sum()
 
                
        return n1
    
    # hard voting
    def votingClassifer_hard(self,estimators,x_test,show_info='percent'):
        
        if show_info not in ["info","percent","nothing"]:
            raise Exception("the attribut 'show_info' should be 'info' or 'percent','nothing'")            
            
        
        cpt=0
        index_classes_glob, class_names_glob, probs_glob=[],[],[]
        N=len(x_test)
        for x in x_test:
            index_classes, class_names, probs=[],[],[]
            for model in estimators:
                img = np.expand_dims(x, axis=0)
                # make a prediction
                y_prob = model.predict(img)[0]
                probabilty = y_prob.flatten()
                max_prob = probabilty.max()
                y_classes = y_prob.argmax(axis=-1)
                index_class, class_name, prob = y_classes,li[y_classes],max_prob
                index_classes.append(index_class)
                class_names.append(class_name)
                probs.append(prob)
            index_classes, class_names, probs = np.array(index_classes), np.array(class_names), np.array(probs)    
            counts = np.bincount(index_classes)
            index_class_prefer=np.argmax(counts)
 
            n1=(index_classes == index_class_prefer).sum()
    
            if n1 == 1:
                print("\n Each estimator predict a different class")
                prob = probs.max()
                indice = [i for i, value in enumerate(probs) if value == prob][0]
                class_name = class_names[indice]

                
            elif n1 == len(estimators)/2 and len(estimators)/2 == self.getNumberElse(index_classes, index_class_prefer):
                
                print("\n the half-estimators predict a class and the other estimators predict a different class")
                
                indices1 = [i for i, value in enumerate(index_classes) if value == index_class_prefer]
                sum2=0
                for ind in indices1:
                    sum2+=probs[ind]
                
                prob1=sum2/len(indices1)
                
                n2=self.getNumberDiff(index_classes, index_class_prefer)      
                
                indices2 = [i for i, value in enumerate(index_classes) if value == n2]
                sum2=0
                for ind in indices2:
                    sum2+=probs[ind]
                
                prob2=sum2/len(indices2)
                
                if prob1 < prob2:
                    prob=prob2
                    indice = [i for i, value in enumerate(index_classes) if value == n2][0]
                    class_name = class_names[indice]
                    
                else:
                    prob=prob1
                    indice = [i for i, value in enumerate(index_classes) if value == index_class_prefer][0]
                    class_name = class_names[indice]
                
            else:
                
                sum1=0
                nbr=0
                for i, index in zip(range(len(probs)),index_classes):
                    if index_class_prefer== index:
                        sum1+=probs[i]
                        nbr+=1
                        
                prob=sum1/nbr
                indice = index_class_prefer
                class_name = li[index_class_prefer]

            
            if show_info=="info":
                cpt+=1
                print("\rpercent: {:.2f}%, li[{}]:{} --> {}".format(cpt*100/N,index_class_prefer, li[index_class_prefer], prob), end='')   
            elif show_info=="percent":
                cpt+=1
                print("\rpercent: {:.2f}%".format(cpt*100/N), end='')
          
                
            index_classes_glob.append(indice)
            class_names_glob.append(class_name)
            probs_glob.append(prob)

        return np.array(index_classes_glob), np.array(class_names_glob), np.array(probs_glob) 
    
   # soft voting 
    def votingClassifer_soft(self, estimators, x_test, weight=None, show_info="percent"):
        
        if show_info not in ["info","percent","nothing"]:
            raise Exception("the attribut 'show_info' should be 'info' or 'percent','nothing'")  

        if weight is None :
            weight=np.ones(len(estimators))

        if len(weight) != len(estimators):
            raise Exception("number of models and wheight should be equals")    

        cpt=0 
        # get number of classes 
        x = image.img_to_array(x_test[0])
        x = np.expand_dims(x, axis=0)
        y_prob = estimators[0].predict(x) 
        num_classes =  y_prob.shape[1]

        Tab=np.zeros(num_classes)# num_classes
        index_classes_glob, class_names_glob, probs_glob=[],[],[]
        N=len(x_test)
        for x in x_test:
            index_classes, class_names, probs= [], [], []
            for model in estimators:
                img = np.expand_dims(x, axis=0)
                # make a prediction
                y_prob = model.predict(img)[0]
                idxs = np.argsort(y_prob)
                # loop over the indexes of the high confidence class labels
                for (index, value) in enumerate(idxs):
                    # build the label and draw the label on the image
                    #label = "{}) {}[{}]: {:.2f}%".format(index,li[value],value, y_prob[value] * 100)
                    #print(label)
                    Tab[value]=y_prob[value]

                probs.append(Tab)

            probs=np.array(probs) # probs.shape: (3, 38)

            proba=[]   

            div=sum(weight)
            for i in range(len(probs[0])):
                s=0
                for j in range(len(weight)):
                    s+= probs[j][i] * weight[j] 
                s=s/div
                proba.append(s)

            proba=np.array(proba)
            max_prob=max(proba)

            indices=[i for i, value in enumerate(proba) if value == max_prob]

            index_class, class_name, prob=indices[0],li[indices[0]],max_prob 

            index_classes_glob.append(index_class)
            class_names_glob.append(class_name)
            probs_glob.append(prob) 

            if show_info=="info":
                cpt+=1
                print("percent: {:.2f}%, li[{}]:{} --> {}%".format(cpt*100/N,index_class, li[index_class], prob))   
            elif show_info=="percent":
                cpt+=1
                print("\rpercent: {:.2f}%".format(cpt*100/N), end='')



        return np.array(index_classes_glob), np.array(class_names_glob), np.array(probs_glob)  

In [ ]:
from sklearn.base import is_classifier, is_regressor
models_names=['model1','model2','model3']
print("model name\t estimator name\t is_regressor\t is_classifier")    
for estimator , model_name in zip([model1,model2,model3],models_names):
        print("{}\t   {}     \t {}      \t {}".format(model_name,estimator.__class__.__name__,
                                              is_regressor(estimator),
                                              is_classifier(estimator)
                                              ))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import numpy as np
models=[model1,model2,model3]
model_names=["model1","model2",'model3']
for model,model_name in zip(models,model_names):
            
    y_prob = model.predict(x_test)
    y_pred1 = y_prob.argmax(axis=-1)
    y_test1=np.argmax(y_test, axis=1)
    # accuracy
    print(model_name+" accuracy: ",accuracy_score(y_test1,y_pred1))
    del model , y_pred1, y_test1
    #Precision
    #print('Precision:', precision_score(y_test,y_pred1))
    #Recall
    #print('Recall:', recall_score(y_test1,y_pred1))
    #F-measure
    #print("Classification report")
    #print(classification_report(y_test1, y_pred1))

In [ ]:
# valid_set
x_test, y_test = valid_set.next() 

#VotingClassifier(hard)
estimators = [model1,model2,model3]
vc=votingClassifer(estimators=estimators,mode="hard",show_info="percent")
index_classes, class_names, probs=vc.predict(x_test)

accuracy

In [ ]:
# inverse_to_categorical inverser format binary to format indexation
# datagenerator use the methode to_categorical for labelsation to frmat binary 
# the method inverce of to_categorical is argmax
import numpy as np
y_test1=np.argmax(y_test, axis=1)
y_pred1=index_classes

# accuracy
from sklearn.metrics import accuracy_score
print("votingClassifer(hard) accuracy : ",accuracy_score(y_test1,y_pred1))